# Treinamento YOLO para semáforos (Bosch)
Este notebook guia o download, conversão e treinamento do YOLOv8 usando o dataset Bosch Small Traffic Lights.

In [ ]:
!pip install --upgrade ultralytics==8.3.28 roboflow tqdm

## Download e extração do dataset
Ajuste `DATA_URL` se você tiver o dataset salvo em outro local (ex.: Google Drive).

In [ ]:
import os, zipfile, json, shutil
from pathlib import Path

BASE_DIR = Path('/content/traffic-lights-bosch')
BASE_DIR.mkdir(parents=True, exist_ok=True)
DATA_URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59812bc9_bosch-small-traffic-lights-dataset/bosch-small-traffic-lights-dataset.zip'

!wget -O {BASE_DIR}/bosch-small-traffic-lights-dataset.zip {DATA_URL}

with zipfile.ZipFile(BASE_DIR/'bosch-small-traffic-lights-dataset.zip','r') as z:
    z.extractall(BASE_DIR)

## Conversão para YOLO
Converta o JSON de anotações para o formato YOLO com classes red/yellow/green/off.

In [ ]:
SRC = BASE_DIR/'rgb_train'
labels_json = BASE_DIR/'train.json'
YOLO = BASE_DIR/'yolo'
YOLO.mkdir(exist_ok=True)
(YOLO/'images').mkdir(exist_ok=True)
(YOLO/'labels').mkdir(exist_ok=True)

class_map = {'Red':0,'Yellow':1,'Green':2,'off':3,'off_or_unknown':3}

def yolo_line(box, w, h):
    xc = (box['x_min'] + box['x_max'])/2/w
    yc = (box['y_min'] + box['y_max'])/2/h
    bw = (box['x_max'] - box['x_min'])/w
    bh = (box['y_max'] - box['y_min'])/h
    return f"{box['cls']} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n"

with open(labels_json) as f:
    data = json.load(f)

for item in data['frames']:
    img = item['filename']
    img_path = SRC/img
    if not img_path.exists():
        continue
    w, h = item['width'], item['height']
    lines = []
    for ann in item['annotations']:
        cls_name = ann['label']
        if cls_name not in class_map:
            continue
        ann_box = {
            'x_min': ann['x_min'], 'x_max': ann['x_max'],
            'y_min': ann['y_min'], 'y_max': ann['y_max'],
            'cls': class_map[cls_name]
        }
        lines.append(yolo_line(ann_box, w, h))
    if not lines:
        continue
    shutil.copy(img_path, YOLO/'images'/img_path.name)
    with open(YOLO/'labels'/f"{img_path.stem}.txt",'w') as lf:
        lf.writelines(lines)

## Split e arquivo data.yaml
Crie pastas train/val/test e um arquivo de config para o YOLO.

In [ ]:
from sklearn.model_selection import train_test_split
images = sorted((YOLO/'images').glob('*.png'))
train_imgs, valtest = train_test_split(images, test_size=0.3, random_state=42, shuffle=True)
val_imgs, test_imgs = train_test_split(valtest, test_size=0.33, random_state=42)


def move_split(split_name, split_imgs):
    (YOLO/split_name/'images').mkdir(parents=True, exist_ok=True)
    (YOLO/split_name/'labels').mkdir(parents=True, exist_ok=True)
    for img in split_imgs:
        lbl = YOLO/'labels'/f"{img.stem}.txt"
        shutil.move(img, YOLO/split_name/'images'/img.name)
        shutil.move(lbl, YOLO/split_name/'labels'/lbl.name)

move_split('train', train_imgs)
move_split('val', val_imgs)
move_split('test', test_imgs)

data_yaml = f"""
path: {YOLO}
train: train/images
val: val/images
test: test/images

names:
  0: red
  1: yellow
  2: green
  3: off
"""
with open(YOLO/'data.yaml','w') as f:
    f.write(data_yaml)
print(data_yaml)

## Treinamento
Ajuste épocas, batch e resolução conforme a GPU disponível.

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
results = model.train(data=YOLO/'data.yaml', epochs=50, imgsz=640, batch=16, device=0, lr0=5e-3, lrf=0.1, mosaic=0.5)

## Avaliação e amostras
Use as métricas e previsões para inspecionar qualidade.

In [ ]:
metrics = model.val(data=YOLO/'data.yaml', imgsz=640)
print(metrics.results_dict)
model.predict(source=YOLO/'test/images', save=True, conf=0.25, iou=0.5)

## Exportar e salvar no Drive
Guarde o checkpoint para reutilizar em produção.

In [ ]:
best_path = model.ckpt_path
reloaded = YOLO(best_path)
reloaded.export(format='onnx', imgsz=640)

from google.colab import drive
drive.mount('/content/drive')
shutil.copy(best_path, '/content/drive/MyDrive/traffic-lights/best.pt')